<a href="https://colab.research.google.com/github/IamAtreus/00-Tensorflow/blob/main/Food_project_pure_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import zipfile

# Download zip file of pizza_steak images

! gdown 1zqeBlp0D9US7b0Jf6zyDzJMV6w73ThCv

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("data.zip", "r")
zip_ref.extractall()
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1zqeBlp0D9US7b0Jf6zyDzJMV6w73ThCv
To: /content/data.zip
100% 296M/296M [00:06<00:00, 48.4MB/s]
